In [1]:
import duckdb
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', None)

In [2]:

# Path to DuckDB database
DB_PATH = '../../database/clinisys_all.duckdb'

use_schema = 'silver'

# List of relevant silver tables
silver_tables = [
    'view_micromanipulacao',
    'view_congelamentos_ovulos',
    'view_descongelamentos_ovulos',
    'view_congelamentos_embrioes',
    'view_embrioes_congelados',
    'view_descongelamentos_embrioes',
    'view_tratamentos',
    # 'view_micromanipulacao_oocitos',
]

# Table that needs special handling
micromanip_oocitos_table = 'view_micromanipulacao_oocitos'


# Prompt for prontuario
prontuario = '161746' #'160751' #'171805'

# Connect to DuckDB
db = duckdb.connect(DB_PATH)

# Special handling for micromanipulacao_oocitos
print(f'\n--- Records from {use_schema}.{micromanip_oocitos_table} for prontuario={prontuario} (via view_micromanipulacao) ---')
try:
    # Get all codigo_ficha for this prontuario
    codigos = db.execute(f"""
        SELECT codigo_ficha FROM {use_schema}.view_micromanipulacao WHERE prontuario = ?
    """, [prontuario]).fetchall()
    codigo_list = [row[0] for row in codigos]
    if codigo_list:
        # Query oocitos table for all id_micromanipulacao in codigo_list
        placeholders = ','.join(['?']*len(codigo_list))
        query = f"""
            SELECT * FROM {use_schema}.{micromanip_oocitos_table}
            WHERE id_micromanipulacao IN ({placeholders})
            ORDER BY id_micromanipulacao, id
        """
        df = db.execute(query, codigo_list).df()
        if not df.empty:
            display(df)
        else:
            print('No records found.')
    else:
        print('No matching codigo_ficha found in view_micromanipulacao.')
except Exception as e:
    print(f'Error querying {use_schema}.{micromanip_oocitos_table}:', e)


for table in silver_tables:
    print(f'\n--- Records from {use_schema}.{table} for prontuario={prontuario} ---')
    # Some tables may not have prontuario, so try/except
    try:
        df = db.execute(f"""
            SELECT * FROM {use_schema}.{table}
            WHERE prontuario = ?
            order by 1
        """, [prontuario]).df()
        if not df.empty:
            print(f'Shape: {df.shape}')
            display(df)
        else:
            print('No records found.')
    except Exception as e:
        print(f'Error querying {use_schema}.{table}:', e)

db.close()


--- Records from silver.view_micromanipulacao_oocitos for prontuario=161746 (via view_micromanipulacao) ---


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,264061,26414,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,e7b840a94422142d5e5e35c8855f71ef,2025-07-21 21:40:02,1,1
1,264062,26414,Não,MII,None,None,3548,None,Transferido,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,9,2,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 4,A,B,None,None,None,None,None,None,None,None,None,None,None,None,9b8d3cc52eddd21f1b2bdc8c2145a868,2025-07-21 21:40:02,1,2
2,264063,26414,Não,MII,None,None,3548,None,Criopreservado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,6,3,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 4,B,C,None,None,None,None,None,None,None,None,None,None,None,Sim,3565e437f70920d0029029dac3df1930,2025-07-21 21:40:02,1,3
3,264064,26414,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,5,3,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,8f66434f432a1647deec5b0280e4f858,2025-07-21 21:40:02,1,4
4,264065,26414,Não,MII,None,None,3548,None,Criopreservado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,10,1,Compactando,None,None,None,None,None,None,Sim,None,Blastocisto Grau 5,B,B,None,None,None,None,None,None,None,None,None,None,None,None,23bded1af307044e2e8df43549c801dc,2025-07-21 21:40:02,1,5
5,264066,26414,Não,MII,None,None,3548,None,Criopreservado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,6,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 5,B,C,None,None,None,None,None,None,None,b1a67631c8bddbb5222ced55261024ae,2025-07-21 21:40:02,1,6
6,264067,26414,Não,MII,None,None,3548,None,Descartado,None,None,None,None,1PN 2CP,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,e315b7d219e6dbd8c2cfa7870ce7f5df,2025-07-21 21:40:02,1,7
7,264068,26414,Não,DEG,None,None,None,None,Descartado,None,None,None,None,None,None,None,None,Não,None,Descongelado OR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9f10f0aa6711ad7af6b22624580ebae2,2025-07-21 21:40:02,0,<NA>
8,264069,26414,Não,DEG,None,None,None,None,Descartado,None,None,None,None,None,None,None,None,Não,None,Descongelado OR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,920556b7f857219c5db85a28fce3036f,2025-07-21 21:40:02,0,<NA>



--- Records from silver.view_micromanipulacao for prontuario=161746 ---
Shape: (1, 211)


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,26414,ED225/25,161746,37,36,2025-07-12,None,11:25,11:35,None,8,None,None,"Sim, descongelada",None,None,None,None,08:00,None,08:00,08:00,08:00,None,None,9,None,None,None,None,None,Sim,DO400/25,None,None,None,2025-07-12,ICSI,07 espermatozoides móveis progressivos\r\n,07 espermatozoides normais\r\n,None,None,None,None,None,None,0000000001,<NA>,<NA>,<


--- Records from silver.view_congelamentos_ovulos for prontuario=161746 ---
No records found.

--- Records from silver.view_descongelamentos_ovulos for prontuario=161746 ---
Shape: (1, 19)


...


--- Records from silver.view_congelamentos_embrioes for prontuario=161746 ---
Shape: (1, 47)


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,29521,E1845/25,1,161746,esposa,2025-07-18,1900-01-01 14:22:00,ED225/25,Não,None,None,None,<NA>,NaT,<NA>,NaT,3,3,E22937,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Ana Reis,Izadora Reis,Cobrar,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,04957593550f8b0b79d553ee79fa26b7,2025-07-21 21:38:47



--- Records from silver.view_embrioes_congelados for prontuario=161746 ---
Shape: (3, 65)


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,87620,264063,29521,0,161746,P3,E22937,<NA>,(3),None,None,None,Blastocisto Grau 4 - B - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,NÃO BIOPSIADO,None,None,None,D5,None,AM,6,E22937,None,<NA>,0da325f3a60d728e059f598c52499488,2025-07-21 21:38:57
1,87621,264065,29521,0,161746,P5,E22937,<NA>,(5),None,None,None,Blastocisto Grau 5 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,NÃO BIOPSIADO,None,None,None,D5,None,AM,6,E22937,None,<NA>,74bc0e258ea1d3ff5e0940b492c1fc8f,2025-07-21 21:38:57
2,87622,264066,29521,0,161746,P6,E22937,<NA>,(6),None,None,None,Blastocisto Grau 5 - B - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,NÃO BIOPSIADO,None,None,None,D6,None,AM,6,E22937,None,<NA>,3ac628d58c379061214ad189e0aa9779,2025-07-21 21:38:57



--- Records from silver.view_descongelamentos_embrioes for prontuario=161746 ---
No records found.

--- Records from silver.view_tratamentos for prontuario=161746 ---
No records found.
